# __Tensorboard et callback__
---

## __Les Callback__
Les [Callback](https://www.tensorflow.org/guide/keras/custom_callback) sont des moyens de customiser le comportement d'un modèle durant les phases d'évaluations.
<br/><br/>
Dans notre cas, nous avons décidé de les utiliser pour :
- Sauvegarder des informations pour [Tensorboard](#Tensorboard).
- Sauvegardes des poids.
- Stopper l'entrainement (Early stopping).
- Création de matrice de confusion.

In [1]:
import tensorflow as tf
import datetime
from tensorflow import keras

### *Sauvegarder des informations pour Tensorboard*
Le callback [TensorBoard](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard) vas enregistrer des informations qui vont pemettre à Tenserboard de construire les courbes de progressions de la précision et de la perte.

In [2]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir = f"logs/fit/{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}",
    histogram_freq = 1
)

### *Sauvegardes des poids*
Le callback [ModelCheckpoint](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint) garde en mémoire le poids des modèles. Ils pourront donc être rechargés sur un modèle de même structure à la prochaine exécutions. 

In [3]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = "logs/weight/cp.ckpt",
    monitor = "val_loss",
    save_weights_only = True,
    save_best_only = True,
    verbose = 1
)

### *Stopper l'entrainement (Early stopping)*
Le callback [EarlyStopping](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping) va, afin d'éviter le sur-apprentissage, limiter le nombre d'époques. Dès qu'il n'y a pas de progrès pendant un nombre d'époques supérieur à la patience, on se trouve face à un sur-apprentissage et on arrête l'exécution.

In [4]:
early_stop_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=3,
    verbose=1,
    mode='auto',
    baseline=None,
    restore_best_weights=False
)

### *Création de matrice de confusion*
Ici, on construit une matrice de confusion entre chaque époque avec sklearn qui est convertie en png pour la sauvegarder puis l'afficher dans Tensorboard.

Code basé sur le travail de Vincent Havard.

In [5]:
#fonction utiles
def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    return image


def plot_confusion_matrix(cm_main, class_names, figsize=(5, 5), accuracy_score=None):
    #Crée un matplotlib plot à partir d'une matrice de confusion numérique et des labels associée
    figure = plt.figure(figsize=figsize)
    ax = plt.imshow(cm_main, interpolation='nearest', cmap=plt.cm.Blues)
    str_title = "Confusion matrix"
    if accuracy_score is not None:
        str_title = str_title + f" acc:{np.round(accuracy_score*100, decimals=3)}%"
    plt.title(str_title)
    
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)



    threshold = 0.5
    # write labels
    for i, j in itertools.product(range(cm_main.shape[0]), range(cm_main.shape[1])):
        val = cm_main[i, j]
        if val >= 0:
            color = "white" if cm_main[i, j] > threshold else "black"
            plt.text(j, i, cm_main[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


def model_to_confusion_matrix(model, x_test, y_test, class_names, plotIt = True):

    y_pred_onehot = model.predict(x_test)
    y_pred = np.argmax(y_pred_onehot, axis=1)
    # Calculate the confusion matrix.
    cm_norm = None
    figure = None
    if y_test.ndim > 1:
        y_test = y_test.squeeze()

    if y_pred.ndim > 1:
        y_pred = y_pred.squeeze()

    idx = np.unique(np.concatenate((y_test, y_pred)))

    cm = sklearn.metrics.confusion_matrix(y_test, y_pred)

    acc = sklearn.metrics.accuracy_score(y_test, y_pred)
    # compute normalzed confusion matrix
    cm_norm = sklearn.metrics.confusion_matrix(y_test, y_pred, normalize="true")
    cm_norm = np.round(cm_norm, decimals=2)
    # print("cm_norm", cm_norm)
    if plotIt:
        cm_main= cm_norm
        # Log the confusion matrix as an image summary.
        n=len(class_names)
        figure = plot_confusion_matrix(cm_main, class_names, figsize=(n,n), accuracy_score=acc)
    # cm, y_pred, y_pred_onehot, (optional: cm_norm, figure)
    return cm, y_pred, y_pred_onehot, cm_norm, figure

In [6]:
#définition du callback perso
class ConfusionMatrixCallback(keras.callbacks.Callback):
    def __init__(self, x_test, y_test, class_names=None, file_writer_cm = tf.summary.create_file_writer('logs/cm')):
        
        self.x_test = x_test
        self.y_test = y_test
        self.class_names = class_names if class_names is not None else np.unique(y_test)
        self.file_writer_cm = file_writer_cm
        
    def on_epoch_end(self, epoch, logs=None):
        _, _, _, _, figure = model_to_confusion_matrix(self.model, self.x_test,self.y_test,self.class_names)
        # Log the confusion matrix as an image summary.
        cm_image = plot_to_image(figure)

        # Log the confusion matrix as an image summary.
        with self.file_writer_cm.as_default():
            
            tf.summary.image("Confusion Matrix", cm_image, step=epoch)
            self.file_writer_cm.flush()
            print(self.file_writer_cm)

2022-10-06 10:20:55.562458: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-06 10:20:55.588953: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-06 10:20:55.589113: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-06 10:20:55.590080: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

### *Faire appels aux Callback*
```python
model.fit(
  X,
  validation_data=Y,
  epochs=epochs,
  callbacks=[
      tensorboard_callback,
      cp_callback,
      confusionMatrix_callback,
      early_stop_callback
  ]
)
```

## __Utilisation de Tensorboard__
Tensorboard est utilisé afin d'afficher et sauvegarde-les statistiques des modèles lors de l'entrainement. Dans notre cas, nous allons récupérer plusieurs informations pour construire les courbes de progressions de la précision et de la perte ainsi que les matrices de confusion à la fin de chaque époque.
<br /><br />
Toutes ces informations seront sauvegardées dans un dossier log par l'intermédiaire de fonction de callback dont on peut fournir une liste lors de l'entrainement d'un modèle.

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

#Problème sur windows avec l'utilisation. Si lors du lancement l'affichage ne se fait pas et qu'il affiche un tensorboard
#  existant datant de la session précédente utilisé les 2 commandes suivantes en ligne de commande
#- taskkill /im tensorboard.exe /f
#- del /q %TMP%\.tensorboard-info\*

In [ ]:
# changez le chemin du dossier après logdir si besoin
%tensorboard --logdir logs --host localhost

#vous pouvez l'ouvrir dans une autre fenêtre avec l'adrresse
# http://localhost:6006